In [1]:

#!pwsh
# 1. Build the project
dotnet build

# Pack up the NuGet package. Note, you should increment the version because the previous one, once installed, will be in your NuGet cache
dotnet pack /p:PackageVersion=1.0.1

# 3. Check that the package is there
Get-ChildItem -Recurse Microsoft.ML.AutoML.Interactive*.nupkg

In [1]:
#i nuget:C:\dev\RevisedTaxiNotebook\Microsoft.ML.AutoML.Interactive\bin\Debug\
#r "nuget:Microsoft.ML.AutoML.Interactive,1.0.1"

Restore sources C:\dev\RevisedTaxiNotebook\Microsoft.ML.AutoML.Interactive\bin\Debug\ Installed Packages Microsoft.ML.AutoML.Interactive, 1.0.1

Loading extensions from `Microsoft.ML.AutoML.Interactive.dll`

In [1]:

/* ML.NET Model Builder generated Notebook file. Notebook files contain both code snippets and rich text elements.
Use the "run" button in the left margin to execute each code snippet and explore ML.NET. */

#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"
#i "nuget:https://mlnetcli.blob.core.windows.net/mlnetcli/index.json"

#r "nuget:Microsoft.ML.AutoML,0.20.3-dev.22174.1"
#r "nuget:Microsoft.Data.Analysis,0.20.3-dev.22174.1"

Restore sources C:\dev\RevisedTaxiNotebook\Microsoft.ML.AutoML.Interactive\bin\Debug\ https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.3-dev.22174.1 microsoft.ml.automl, 0.20.3-dev.22174.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [1]:

// Import common usings.
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.SearchSpace;
using Microsoft.ML.SearchSpace.Tuner;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.AutoML;
using System;
using System.Collections.Generic;
using System.Text;
using System.Text.Json;
using Microsoft.ML.Runtime;
using Microsoft.ML.AutoML.Interactive;

In [1]:
var TrainDataPath = @"C:\dev\datasets\titanic.csv";;

var context = new MLContext();
var df = DataFrame.LoadCsv(TrainDataPath);

var pipeline = context.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"Sex", @"Sex"), new InputOutputColumnPair(@"Embarked", @"Embarked") })
                           .Append(context.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"Pclass", @"Pclass"), new InputOutputColumnPair(@"Age", @"Age"), new InputOutputColumnPair(@"SibSp", @"SibSp"), new InputOutputColumnPair(@"Parch", @"Parch"), new InputOutputColumnPair(@"Fare", @"Fare") }))
                           .Append(context.Transforms.Concatenate(@"TextFeature", @"Name", "Ticket", "Cabin"))
                           .Append(context.Transforms.Text.FeaturizeText("TextFeature", "TextFeature"))
                           .Append(context.Transforms.Concatenate(@"Features", new[] { @"Sex", @"Embarked", @"Pclass", @"Age", @"SibSp", @"Parch", @"Fare", "TextFeature" }))
                           .Append(context.Transforms.Conversion.ConvertType("Survived", "Survived", DataKind.Boolean))
						   .Append(context.Auto().BinaryClassification(labelColumnName: "Survived", useFastForest: false));

var trainTestSplit = context.Data.TrainTestSplit(df, 0.1);
var monitor = new NotebookMonitor();

            var experiment = context.Auto().CreateExperiment()
                                .SetPipeline(pipeline)
                                .SetTrainingTimeInSeconds(150)
                                .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
                                .SetEvaluateMetric(BinaryClassificationMetric.Accuracy, "Survived", "PredictedLabel")
                                .SetMonitor(monitor);
			
			monitor.SetUpdate(monitor.Display());
            var res = experiment.Run().Result;

			

Trial,Metric,Pipeline
0,0.6794871794871795,Unknown=>SdcaLogisticRegressionBinary
1,0.5512820512820513,Unknown=>SdcaLogisticRegressionBinary
2,0.8205128205128205,Unknown=>LbfgsLogisticRegressionBinary
3,0.8205128205128205,Unknown=>FastTreeBinary
4,0.8205128205128205,Unknown=>LightGbmBinary
5,0.7948717948717948,Unknown=>LbfgsLogisticRegressionBinary
6,0.7307692307692307,Unknown=>SdcaLogisticRegressionBinary
7,0.7948717948717948,Unknown=>LightGbmBinary
8,0.8333333333333334,Unknown=>FastTreeBinary
9,0.717948717948718,Unknown=>SdcaLogisticRegressionBinary
